In [87]:
import pandas as pd
import seaborn as sns
import pickle


import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

* Download the Data using the command line terminal wget
* Referal to learn with Machine Learning Zoomcamp 2:23
* Read Data and understanding different columns of the data using df.head and data dictionary 3:06
* * 
* Calulate the duration of each trip fropoff - pickup time.
* * convert value to pandas date time and calculate the difference in value
* * Prprocess the created duration column into minutes
* * Visuale distribution and data describe of duration 9:50
* * Filter duration that is within 1 and 60 mins  13:10
* * Specify catergorical and numeric variables
* if you do not have pyarrow installed or fastparquet, you will have to install one
** 

In [76]:
data_Jan_2021 = 'data/green_tripdata_2021-01.parquet'
data_Feb_2021 = 'data/green_tripdata_2021-02.parquet'

In [77]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df ['duration'] = pd.to_datetime(df['lpep_dropoff_datetime']) - pd.to_datetime(df['lpep_pickup_datetime'])
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
 
    df = df[((df.duration>=1) & (df.duration<=60))]

    categorical = ['DOLocationID', 'PULocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [78]:
df_train = read_dataframe(data_Jan_2021)
df_val = read_dataframe(data_Feb_2021)

In [79]:
len(df_train), len(df_val)

(73908, 61921)

In [92]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


numerical = ['trip_distance']
categorical = ['PU_DO']#['DOLocationID', 'PULocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)

In [44]:
# ! conda install -c conda-forge fastparquet
# ! conda install -c conda-forge pyarrow


In [93]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [94]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)


4.6408189854931665

In [97]:
with open ('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr),f_out)

In [95]:
# lr = Lasso(alpha=0.001)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_train) 
# mean_squared_error(y_train, y_pred, squared=False)

8.345478908061276

In [ ]:
# sns.displot(y_pred, label='Prediction')
# sns.distplot(y_train, label='Real')
# plt.legend()